# 导入相关库

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# 读数据

In [2]:
data = pd.read_excel('./2023年中国研究生数学建模竞赛赛题/E题/data3b.xlsx')
data_image = pd.read_excel('./2023年中国研究生数学建模竞赛赛题/E题/竞赛发布数据/表3-患者影像信息血肿及水肿的形状及灰度分布.xlsx')

In [3]:
data = data.rename(columns={'入院首次影像检查流水号':'流水号'})

In [4]:
data[['流水号', '随访1流水号', '随访2流水号', '随访3流水号', '随访4流水号', '随访5流水号', '随访6流水号', '随访7流水号', '随访8流水号']] = data[['流水号', '随访1流水号', '随访2流水号', '随访3流水号', '随访4流水号', '随访5流水号', '随访6流水号', '随访7流水号', '随访8流水号']].fillna(0)

In [5]:
date_dict = {'流水号': ['HM_volume', 'HM_ACA_R_Ratio', 'HM_MCA_R_Ratio', 'HM_PCA_R_Ratio', 
                        'HM_Pons_Medulla_R_Ratio', 'HM_Cerebellum_R_Ratio', 'HM_ACA_L_Ratio', 
                        'HM_MCA_L_Ratio', 'HM_PCA_L_Ratio', 'HM_Pons_Medulla_L_Ratio', 
                        'HM_Cerebellum_L_Ratio', 'ED_volume', 'ED_ACA_R_Ratio', 'ED_MCA_R_Ratio', 
                        'ED_PCA_R_Ratio', 'ED_Pons_Medulla_R_Ratio', 'ED_Cerebellum_R_Ratio', 
                        'ED_ACA_L_Ratio', 'ED_MCA_L_Ratio', 'ED_PCA_L_Ratio', 
                        'ED_Pons_Medulla_L_Ratio', 'ED_Cerebellum_L_Ratio'], 
             '随访1流水号': ['HM_volume.1', 'HM_ACA_R_Ratio.1', 'HM_MCA_R_Ratio.1', 'HM_PCA_R_Ratio.1', 
                        'HM_Pons_Medulla_R_Ratio.1', 'HM_Cerebellum_R_Ratio.1', 'HM_ACA_L_Ratio.1', 
                        'HM_MCA_L_Ratio.1', 'HM_PCA_L_Ratio.1', 'HM_Pons_Medulla_L_Ratio.1', 
                        'HM_Cerebellum_L_Ratio.1', 'ED_volume.1', 'ED_ACA_R_Ratio.1', 'ED_MCA_R_Ratio.1', 
                        'ED_PCA_R_Ratio.1', 'ED_Pons_Medulla_R_Ratio.1', 'ED_Cerebellum_R_Ratio.1', 
                        'ED_ACA_L_Ratio.1', 'ED_MCA_L_Ratio.1', 'ED_PCA_L_Ratio.1', 
                        'ED_Pons_Medulla_L_Ratio.1', 'ED_Cerebellum_L_Ratio.1'], 
             '随访2流水号': ['HM_volume.2', 'HM_ACA_R_Ratio.2', 'HM_MCA_R_Ratio.2', 'HM_PCA_R_Ratio.2', 
                        'HM_Pons_Medulla_R_Ratio.2', 'HM_Cerebellum_R_Ratio.2', 'HM_ACA_L_Ratio.2', 
                        'HM_MCA_L_Ratio.2', 'HM_PCA_L_Ratio.2', 'HM_Pons_Medulla_L_Ratio.2', 
                        'HM_Cerebellum_L_Ratio.2', 'ED_volume.2', 'ED_ACA_R_Ratio.2', 'ED_MCA_R_Ratio.2', 
                        'ED_PCA_R_Ratio.2', 'ED_Pons_Medulla_R_Ratio.2', 'ED_Cerebellum_R_Ratio.2', 
                        'ED_ACA_L_Ratio.2', 'ED_MCA_L_Ratio.2', 'ED_PCA_L_Ratio.2', 
                        'ED_Pons_Medulla_L_Ratio.2', 'ED_Cerebellum_L_Ratio.2'], 
             '随访3流水号': ['HM_volume.3', 'HM_ACA_R_Ratio.3', 'HM_MCA_R_Ratio.3', 'HM_PCA_R_Ratio.3', 
                        'HM_Pons_Medulla_R_Ratio.3', 'HM_Cerebellum_R_Ratio.3', 'HM_ACA_L_Ratio.3', 
                        'HM_MCA_L_Ratio.3', 'HM_PCA_L_Ratio.3', 'HM_Pons_Medulla_L_Ratio.3', 
                        'HM_Cerebellum_L_Ratio.3', 'ED_volume.3', 'ED_ACA_R_Ratio.3', 'ED_MCA_R_Ratio.3', 
                        'ED_PCA_R_Ratio.3', 'ED_Pons_Medulla_R_Ratio.3', 'ED_Cerebellum_R_Ratio.3', 
                        'ED_ACA_L_Ratio.3', 'ED_MCA_L_Ratio.3', 'ED_PCA_L_Ratio.3', 
                        'ED_Pons_Medulla_L_Ratio.3', 'ED_Cerebellum_L_Ratio.3'], 
             '随访4流水号': ['HM_volume.4', 'HM_ACA_R_Ratio.4', 'HM_MCA_R_Ratio.4', 'HM_PCA_R_Ratio.4', 
                        'HM_Pons_Medulla_R_Ratio.4', 'HM_Cerebellum_R_Ratio.4', 'HM_ACA_L_Ratio.4', 
                        'HM_MCA_L_Ratio.4', 'HM_PCA_L_Ratio.4', 'HM_Pons_Medulla_L_Ratio.4', 
                        'HM_Cerebellum_L_Ratio.4', 'ED_volume.4', 'ED_ACA_R_Ratio.4', 'ED_MCA_R_Ratio.4', 
                        'ED_PCA_R_Ratio.4', 'ED_Pons_Medulla_R_Ratio.4', 'ED_Cerebellum_R_Ratio.4', 
                        'ED_ACA_L_Ratio.4', 'ED_MCA_L_Ratio.4', 'ED_PCA_L_Ratio.4', 
                        'ED_Pons_Medulla_L_Ratio.4', 'ED_Cerebellum_L_Ratio.4'], 
             '随访5流水号': ['HM_volume.5', 'HM_ACA_R_Ratio.5', 'HM_MCA_R_Ratio.5', 'HM_PCA_R_Ratio.5', 
                        'HM_Pons_Medulla_R_Ratio.5', 'HM_Cerebellum_R_Ratio.5', 'HM_ACA_L_Ratio.5', 
                        'HM_MCA_L_Ratio.5', 'HM_PCA_L_Ratio.5', 'HM_Pons_Medulla_L_Ratio.5', 
                        'HM_Cerebellum_L_Ratio.5', 'ED_volume.5', 'ED_ACA_R_Ratio.5', 'ED_MCA_R_Ratio.5', 
                        'ED_PCA_R_Ratio.5', 'ED_Pons_Medulla_R_Ratio.5', 'ED_Cerebellum_R_Ratio.5', 
                        'ED_ACA_L_Ratio.5', 'ED_MCA_L_Ratio.5', 'ED_PCA_L_Ratio.5', 
                        'ED_Pons_Medulla_L_Ratio.5', 'ED_Cerebellum_L_Ratio.5'], 
             '随访6流水号': ['HM_volume.6', 'HM_ACA_R_Ratio.6', 'HM_MCA_R_Ratio.6', 'HM_PCA_R_Ratio.6', 
                        'HM_Pons_Medulla_R_Ratio.6', 'HM_Cerebellum_R_Ratio.6', 'HM_ACA_L_Ratio.6', 
                        'HM_MCA_L_Ratio.6', 'HM_PCA_L_Ratio.6', 'HM_Pons_Medulla_L_Ratio.6', 
                        'HM_Cerebellum_L_Ratio.6', 'ED_volume.6', 'ED_ACA_R_Ratio.6', 'ED_MCA_R_Ratio.6', 
                        'ED_PCA_R_Ratio.6', 'ED_Pons_Medulla_R_Ratio.6', 'ED_Cerebellum_R_Ratio.6', 
                        'ED_ACA_L_Ratio.6', 'ED_MCA_L_Ratio.6', 'ED_PCA_L_Ratio.6', 
                        'ED_Pons_Medulla_L_Ratio.6', 'ED_Cerebellum_L_Ratio.6'], 
             '随访7流水号': ['HM_volume.7', 'HM_ACA_R_Ratio.7', 'HM_MCA_R_Ratio.7', 'HM_PCA_R_Ratio.7', 
                        'HM_Pons_Medulla_R_Ratio.7', 'HM_Cerebellum_R_Ratio.7', 'HM_ACA_L_Ratio.7', 
                        'HM_MCA_L_Ratio.7', 'HM_PCA_L_Ratio.7', 'HM_Pons_Medulla_L_Ratio.7', 
                        'HM_Cerebellum_L_Ratio.7', 'ED_volume.7', 'ED_ACA_R_Ratio.7', 'ED_MCA_R_Ratio.7', 
                        'ED_PCA_R_Ratio.7', 'ED_Pons_Medulla_R_Ratio.7', 'ED_Cerebellum_R_Ratio.7', 
                        'ED_ACA_L_Ratio.7', 'ED_MCA_L_Ratio.7', 'ED_PCA_L_Ratio.7', 
                        'ED_Pons_Medulla_L_Ratio.7', 'ED_Cerebellum_L_Ratio.7'], 
             '随访8流水号': ['HM_volume.8', 'HM_ACA_R_Ratio.8', 'HM_MCA_R_Ratio.8', 'HM_PCA_R_Ratio.8', 
                        'HM_Pons_Medulla_R_Ratio.8', 'HM_Cerebellum_R_Ratio.8', 'HM_ACA_L_Ratio.8', 
                        'HM_MCA_L_Ratio.8', 'HM_PCA_L_Ratio.8', 'HM_Pons_Medulla_L_Ratio.8', 
                        'HM_Cerebellum_L_Ratio.8', 'ED_volume.8', 'ED_ACA_R_Ratio.8', 'ED_MCA_R_Ratio.8', 
                        'ED_PCA_R_Ratio.8', 'ED_Pons_Medulla_R_Ratio.8', 'ED_Cerebellum_R_Ratio.8', 
                        'ED_ACA_L_Ratio.8', 'ED_MCA_L_Ratio.8', 'ED_PCA_L_Ratio.8', 
                        'ED_Pons_Medulla_L_Ratio.8', 'ED_Cerebellum_L_Ratio.8']}

In [6]:
feature1 = []
feature2 = []
for i in data.iterrows():
    feat1 = []
    feat2 = []
    i = i[1]
    for num in date_dict:
        if i[num] == 0:
            break
        else:
            check = i[num]
            feat1.append(list(i[date_dict[num]].values))
            image_d = list(data_image.query("流水号 == @check").values.reshape(-1)[1:])
            if len(image_d) != 0:
                feat2.append(image_d)
    feature1.append(np.mean(feat1, axis=0))
    feature2.append(np.mean(feat2, axis=0))

In [7]:
feature1_name = ['HM_volume_mean', 'HM_ACA_R_Ratio_mean', 'HM_MCA_R_Ratio_mean', 'HM_PCA_R_Ratio_mean', 
                 'HM_Pons_Medulla_R_Ratio_mean', 'HM_Cerebellum_R_Ratio_mean', 'HM_ACA_L_Ratio_mean', 
                 'HM_MCA_L_Ratio_mean', 'HM_PCA_L_Ratio_mean', 'HM_Pons_Medulla_L_Ratio_mean', 
                 'HM_Cerebellum_L_Ratio_mean', 'ED_volume_mean', 'ED_ACA_R_Ratio_mean', 'ED_MCA_R_Ratio_mean', 
                 'ED_PCA_R_Ratio_mean', 'ED_Pons_Medulla_R_Ratio_mean', 'ED_Cerebellum_R_Ratio_mean', 
                 'ED_ACA_L_Ratio_mean', 'ED_MCA_L_Ratio_mean', 'ED_PCA_L_Ratio_mean', 
                 'ED_Pons_Medulla_L_Ratio_mean', 'ED_Cerebellum_L_Ratio_mean']

feature2_name = ['original_shape_Elongation_mean',
 'original_shape_Flatness_mean',
 'original_shape_LeastAxisLength_mean',
 'original_shape_MajorAxisLength_mean',
 'original_shape_Maximum2DDiameterColumn_mean',
 'original_shape_Maximum2DDiameterRow_mean',
 'original_shape_Maximum2DDiameterSlice_mean',
 'original_shape_Maximum3DDiameter_mean',
 'original_shape_MeshVolume_mean',
 'original_shape_MinorAxisLength_mean',
 'original_shape_Sphericity_mean',
 'original_shape_SurfaceArea_mean',
 'original_shape_SurfaceVolumeRatio_mean',
 'original_shape_VoxelVolume_mean',
 'NCCT_original_firstorder_10Percentile_mean',
 'NCCT_original_firstorder_90Percentile_mean',
 'NCCT_original_firstorder_Energy_mean',
 'NCCT_original_firstorder_Entropy_mean',
 'NCCT_original_firstorder_InterquartileRange_mean',
 'NCCT_original_firstorder_Kurtosis_mean',
 'NCCT_original_firstorder_Maximum_mean',
 'NCCT_original_firstorder_MeanAbsoluteDeviation_mean',
 'NCCT_original_firstorder_Mean_mean',
 'NCCT_original_firstorder_Median_mean',
 'NCCT_original_firstorder_Minimum_mean',
 'NCCT_original_firstorder_Range_mean',
 'NCCT_original_firstorder_RobustMeanAbsoluteDeviation_mean',
 'NCCT_original_firstorder_RootMeanSquared_mean',
 'NCCT_original_firstorder_Skewness_mean',
 'NCCT_original_firstorder_Uniformity_mean',
 'NCCT_original_firstorder_Variance_mean']

In [8]:
df1 = pd.DataFrame(feature1, columns=feature1_name)
df2 = pd.DataFrame(feature2, columns=feature2_name)

In [9]:
data = pd.concat([data[['ID', '90天mRS', '年龄', '性别', '脑出血前mRS评分', '高血压病史', '卒中病史', '糖尿病史', 
                '房颤史', '冠心病史', '吸烟史', '饮酒史', '发病到首次影像检查时间间隔', '血压', '脑室引流', 
                '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']],
                  df1, df2], axis=1)

In [10]:
train = data.iloc[:100, :]
test = data.iloc[100:, :]

In [11]:
label = train['90天mRS']

In [12]:
label.value_counts()

3.0    20
2.0    20
1.0    19
5.0    15
4.0    12
0.0    10
6.0     4
Name: 90天mRS, dtype: int64

# 特征工程

In [13]:
train.drop(['ID','90天mRS'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)
train['高压'] = train['血压'].apply(lambda x: int(x.split('/')[0]))
train['低压'] = train['血压'].apply(lambda x: int(x.split('/')[1]))
test['高压'] = test['血压'].apply(lambda x: int(x.split('/')[0]))
test['低压'] = test['血压'].apply(lambda x: int(x.split('/')[1]))
train.drop(['血压'], axis=1, inplace=True)
test.drop(['血压'], axis=1, inplace=True)
train['性别'] = train['性别'].map({'男':0, '女':1})
test['性别'] = test['性别'].map({'男':0, '女':1})
feat = train.columns
scaler = StandardScaler()
train[feat] = scaler.fit_transform(train[feat])
test[feat] = scaler.transform(test[feat])
print(len(feat))

73


In [14]:
feat = ['糖尿病史', 'NCCT_original_firstorder_Mean_mean', 'original_shape_Maximum2DDiameterSlice_mean', 'original_shape_MeshVolume_mean', 'original_shape_LeastAxisLength_mean', 'ED_volume_mean', 'original_shape_Sphericity_mean', 'original_shape_MajorAxisLength_mean', 'NCCT_original_firstorder_Range_mean', 'original_shape_Flatness_mean', 'HM_volume_mean', 'NCCT_original_firstorder_Maximum_mean', 'NCCT_original_firstorder_10Percentile_mean', 'original_shape_MinorAxisLength_mean', 'NCCT_original_firstorder_90Percentile_mean', 'original_shape_Maximum2DDiameterColumn_mean']

# 交叉验证

In [15]:
# oof = np.zeros(len(train))
# feat_imp = np.zeros(len(feat))
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
#     print(f'第{i+1}折....')
#     tra_x = train.iloc[tra_index, :][feat].values
#     tra_y = label.iloc[tra_index].values
#     val_x = train.iloc[val_index, :][feat].values
#     val_y = label.iloc[val_index].values
#     model = LogisticRegression(random_state=42)
#     model.fit(tra_x, tra_y)
#     feat_imp += abs(np.mean(model.coef_, axis=0))
#     pred_soft = model.predict_proba(val_x)[:, 1]
#     pred = model.predict(val_x)
#     score = accuracy_score(val_y, pred)
#     print(f'准确率为{score}')
#     oof[val_index] = pred
# oof_score = accuracy_score(label.values, oof)
# feat_imp /= 5
# print('*' * 30)
# print(f'OOF 准确率 = {oof_score}')

In [16]:
def Range_Deviation(y_true, y_pred):
    n = len(y_true)
    m = np.sum(np.abs(y_true - y_pred) <= 1)
    
    return m / n

In [17]:
oof = np.zeros(len(train))
feat_imp = np.zeros(len(feat))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
params = {'booster': 'gbtree', 
         'device': 'cpu', 
         'eta': 0.01, 
         'max_depth': 6, 
         'objective': 'multi:softmax',
          'num_round': 300, 
         'seed': 42, 
         'num_class': 7}
for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
    print(f'第{i+1}折....')
    tra_x = train.iloc[tra_index, :][feat]
    tra_y = label.iloc[tra_index].values
    val_x = train.iloc[val_index, :][feat]
    val_y = label.iloc[val_index].values
    model = xgb.XGBClassifier(params, early_stopping_rounds=30)
    model.fit(tra_x, tra_y, eval_set=[(val_x, val_y)])
    feat_imp += model.feature_importances_
#    pred_soft = model.predict_proba(val_x)[:, 1]
    pred = model.predict(val_x)
    score = Range_Deviation(val_y, pred)
    print(f'准确率为{score}')
    oof[val_index] = pred
oof_score = Range_Deviation(label.values, oof)
feat_imp /= 5
print('*' * 30)
print(f'OOF 准确率 = {oof_score}')

第1折....
[0]	validation_0-mlogloss:1.77405
[1]	validation_0-mlogloss:1.70460
[2]	validation_0-mlogloss:1.68352
[3]	validation_0-mlogloss:1.64450
[4]	validation_0-mlogloss:1.59382
[5]	validation_0-mlogloss:1.57057
[6]	validation_0-mlogloss:1.54075
[7]	validation_0-mlogloss:1.55316
[8]	validation_0-mlogloss:1.54562
[9]	validation_0-mlogloss:1.53422
[10]	validation_0-mlogloss:1.55122
[11]	validation_0-mlogloss:1.55195
[12]	validation_0-mlogloss:1.57778
[13]	validation_0-mlogloss:1.57043
[14]	validation_0-mlogloss:1.55909
[15]	validation_0-mlogloss:1.57835
[16]	validation_0-mlogloss:1.57604
[17]	validation_0-mlogloss:1.59326
[18]	validation_0-mlogloss:1.58935
[19]	validation_0-mlogloss:1.57825
[20]	validation_0-mlogloss:1.56834
[21]	validation_0-mlogloss:1.57249
[22]	validation_0-mlogloss:1.56971
[23]	validation_0-mlogloss:1.56975
[24]	validation_0-mlogloss:1.56690
[25]	validation_0-mlogloss:1.55568
[26]	validation_0-mlogloss:1.55816
[27]	validation_0-mlogloss:1.56992
[28]	validation_0-mlog

# 特征选择

In [18]:
# feature_import = pd.DataFrame()
# feature_import['columns'] = feat
# feature_import['score'] = feat_imp
# feature_import = feature_import.sort_values(by='score', ascending=False)
# best_thre = 0
# best_score = 0
# for j in np.arange(0, 0.03, 0.001):
#     print(f'阈值={j}')
#     feat = list(feature_import[feature_import['score'] >= j]['columns'].values)
#     oof = np.zeros(len(train))
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for i, (tra_index, val_index) in enumerate(skf.split(train, label)):
#         tra_x = train.iloc[tra_index, :][feat]
#         tra_y = label.iloc[tra_index].values
#         val_x = train.iloc[val_index, :][feat]
#         val_y = label.iloc[val_index].values
#         model = xgb.XGBClassifier(params, early_stopping_rounds=30)
#         model.fit(tra_x, tra_y, eval_set=[(val_x, val_y)])
#         pred = model.predict(val_x)
#         score = Range_Deviation(val_y, pred)
#         oof[val_index] = pred
#     oof_score = Range_Deviation(label.values, oof)
#     if oof_score > best_score:
#         best_score = oof_score
#         best_thre = j
#     print(f'OOF 准确率 = {oof_score}')
#     print('*' * 30)

# print(list(feature_import[feature_import['score'] >= best_thre]['columns'].values))
# print(best_score)

# 全数据训练和预测

In [19]:
params = {'booster': 'gbtree', 
         'device': 'cpu', 
         'eta': 0.01, 
         'max_depth': 6, 
         'objective': 'multi:softmax',
          'num_round': 5, 
         'seed': 42, 
         'num_class': 7}
model = xgb.XGBClassifier(params)
model.fit(tra_x, tra_y)
train_pred = model.predict(train[feat])
test_pred = model.predict(test[feat])
all_pred = np.concatenate([train_pred, test_pred])
data['pred'] = all_pred
data[['ID', 'pred']].to_csv('3b.csv', index=None)

In [20]:
#可改进的地方：特征工程，使用多个模型，特征选择，调参，多模型集成